Titre
=====

**Author:** Laurent Siksous



### Imports



In [5]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle

from azureml.core.environment import Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Experiment, Workspace, Dataset
from azureml.opendatasets import MNIST
import mlflow

In [9]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')
experiment_name = 'JoyMNIST-SVC'
exp = Experiment(workspace=ws, name=experiment_name)
env = Environment.get(workspace=ws, name="mlflow-racc00n")

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DLHBPXQCC to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
raccoon	westeurope	racc001_group


In [3]:
cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cpu-cluster")
except ComputeTargetException:
    print("Creating new cpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Creating new cpu-cluster
InProgress...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [10]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment("JoyMNIST-SVC")

## The Data



### Look at the Data



In [11]:
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

mnist_file_dataset = MNIST.get_file_dataset()
mnist_file_dataset.download(data_folder, overwrite=True)

mnist_file_dataset = mnist_file_dataset.register(workspace=ws,
                                                 name='mnist_opendataset',
                                                 description='training and test dataset',
                                                 create_new_version=True)

In [17]:
import glob
import gzip
import struct
import numpy as np

# load compressed MNIST gz files and return numpy arrays
def load_data(filename, label=False):
    with gzip.open(filename) as gz:
        struct.unpack('I', gz.read(4))
        n_items = struct.unpack('>I', gz.read(4))
        if not label:
            n_rows = struct.unpack('>I', gz.read(4))[0]
            n_cols = struct.unpack('>I', gz.read(4))[0]
            res = np.frombuffer(gz.read(n_items[0] * n_rows * n_cols), dtype=np.uint8)
            res = res.reshape(n_items[0], n_rows * n_cols)
        else:
            res = np.frombuffer(gz.read(n_items[0]), dtype=np.uint8)
            res = res.reshape(n_items[0], 1)
    return res

# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = load_data(glob.glob(os.path.join(data_folder,"**/train-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
X_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
y_train = load_data(glob.glob(os.path.join(data_folder,"**/train-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)
y_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)

### Transform the Data



### Split the Data



In [7]:
test_size = 0.9
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

## Training



In [22]:
# start run
run = mlflow.start_run()

# enable automatic logging
mlflow.sklearn.autolog()

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
          'gamma': [0.0001, 0.001, 0.01, 0.1],
          'kernel':['rbf'] }
mlflow.log_params(param_grid)

gm = RandomizedSearchCV(SVC(), param_distributions = param_grid, n_iter = 16, n_jobs=-1, cv=3, verbose=True)
gm.fit(X_train, y_train)

mlflow.log_metric("training_time", train_time)

# print best parameter after tuning
print(gm.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(gm.best_estimator_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/model_selection/_search.py:277: UserWarning: The total space of parameters 24 is smaller than n_iter=72. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


## Evaluation



In [ ]:
score = m.score(X_test, y_test)
y_proba = m.predict(X_test)
y_pred = y_proba.argmax(axis=1)
acc = accuracy_score(y_test, y_pred)

mlflow.log_metrics({"accuracy": acc})

Error: Kernel is dead

In [1]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))


# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

In [21]:
#mlflow.log_artifact("SVC.ipynb")

# end run
mlflow.end_run()